## Libraries + configuration

In [1]:
import sys

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import datetime as dt
import zipfile
import os
from os.path import exists

#from osgeo import gdal
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.merge import merge
import rasterio.mask
from rasterio.features import bounds
from rasterio.enums import Resampling
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import fiona

import json


In [2]:
# dates to search for new Sentinel images 
# e.g. '20211025' 'rrrrMMdd'
startDate = '20220401' # included 
endDate = '20220405' # not included

# you need to specify if the data will be actual or historical 
# so the indexes do not merge together
oblast = 'kk'
indexes = {oblast + 'EVIhist':[],oblast + 'NDVIhist':[],oblast + 'VCIhist':[],oblast + 'WDRVIhist':[]}
# indexes = {oblast + 'EVI':[],oblast + 'NDVI':[],oblast + 'VCI':[],oblast + 'WDRVI':[]}

HOMEdir = os.getcwd()
DATAdir = HOMEdir + r'\data'
RESULTSdir = HOMEdir + r'\results'
SHAREdir = HOMEdir + r'\share'
REPROJdir = HOMEdir + r'\reproject'

In [3]:
cloudCover = (0,30)

## Download / reproject / mask functions

In [4]:
# code changed for dev environment in dataCheck2A
def dataCheck2A():
    """
    Calls Sentinel API to find new 2A products.
    Reguires lastRefresh.txt file in HOMEdir.
    Return dataframe with products found.
    """
    os.chdir(HOMEdir)

    api = SentinelAPI('zuluzi','kom987ik','https://scihub.copernicus.eu/dhus')
    footprint = geojson_to_wkt(read_geojson(HOMEdir + '\\geojsonExtents\\' + oblast + '.geojson'))
    products = api.query(footprint,
# searching from last refresh date (saved in file) till now
                         #date=(dt.datetime.strptime(open('lastRefresh.txt','r').read(), '%Y-%m-%d %H:%M:%S.%f'), dt.datetime.now()),
                         date=(startDate, endDate),
                         platformname='Sentinel-2',
                         processinglevel = 'Level-2A',
                         cloudcoverpercentage=cloudCover)

    products_df = api.to_dataframe(products)

    if products_df.empty:
        print('No new products found')
# update the datetime of last refresh
        fileRefresh = open('lastRefresh.txt','w')
        fileRefresh.write(str(dt.datetime.now()))
        fileRefresh.close()
    else:
        print('Products found: ')
        for product in products_df.iterrows():
            print('Found: ', product[1]['filename'])

    return products_df

def dataDownload2A(products_df):
    """
    Downloads and unzips products found.
    Reguires data returned from dataCheck() function and lastRefresh.txt file in HOMEdir.
    Returns nothing.
    """

    os.chdir(DATAdir)
    api = SentinelAPI('zuluzi','kom987ik','https://scihub.copernicus.eu/dhus')

    for product in products_df.iterrows():
    #product = products_df
        print('Now downloading: ', product[1]['filename']) # filename of the product
        api.download(product[1]['uuid']) # download the product using uuid
        odata = api.get_product_odata(product[1]['uuid'], full=True)
        with zipfile.ZipFile(odata['title']+'.zip',"r") as zip_ref:
            zip_ref.extractall()
            print("Zipped file extracted to", product[1]['filename'], "folder")

    os.chdir(HOMEdir)
def bands2A(product):
    """
    Gets directories to different bands of the 2A product.
    Returns dictionary with directories to the bands.
    """
# getting into exact imagery folder
    os.chdir(DATAdir + '/' + product)

# finding directories for needed bands (only used implemented)
    bands = {'b02_10m':None, 
             'b03_10m':None, 
             'b04_10m':None, 
             'b08_10m':None,
             'b11_20m':None,
             'b12_20m':None,
             'b01_60m':None, 
             'b03_60m':None}

    print('Getting bands directories for', product)
    for root,dirs,files in os.walk(os.getcwd()):
        for file in files:
# finding different bands and resolution
            if file.endswith("_B04_10m.jp2"):
                bands['b04_10m'] = os.path.join(root, file)
            if file.endswith("_B03_10m.jp2"):
                bands['b03_10m'] = os.path.join(root, file)
            if file.endswith("_B02_10m.jp2"):
                bands['b02_10m'] = os.path.join(root, file)
            if file.endswith("_B08_10m.jp2"):
                bands['b08_10m'] = os.path.join(root, file)
            if file.endswith("_B11_20m.jp2"):
                bands['b11_20m'] = os.path.join(root, file)
            if file.endswith("_B12_20m.jp2"):
                bands['b12_20m'] = os.path.join(root, file)
            if file.endswith("_B01_60m.jp2"):
                bands['b01_60m'] = os.path.join(root, file)
            if file.endswith("_B03_60m.jp2"):
                bands['b03_60m'] = os.path.join(root, file)
    os.chdir(HOMEdir)
    return bands
def showBand(bandDir, color_map='gray'):
    """
    Creates a chart with raster values.
    Requires directory for a saved raster file and optionally a color map.
    """
    fig = plt.figure(figsize=(8, 8))
    band = rasterio.open(bandDir).read(1).astype('f4')
    image_layer = plt.imshow(band)
    image_layer.set_cmap(color_map)
    plt.colorbar()
    plt.show()
def epsg3857(product, directory, delete = False):
    """
    Requires name of the .tif file and the directory.
    Saves reprojected to EPSG 3857 (Web Mercator) .tif file in REPROJdir.
    Can delete provided product in the specified directory after reprojection.
    """
    product = product.split('.tif')[0]
    # print('Reprojecting: ' + product)
    dst_crs = 'EPSG:3857'
    with rasterio.open(directory + '\\' + product + '.tif') as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(REPROJdir + '\\' + product + '_reprojected.tif' , 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
    if delete == True:
        if os.path.exists(directory + '\\' + product + '.tif'):
            os.remove(directory + '\\' + product + '.tif')
            # print('    Deleted: ' + product + '.tif')
def mergeWQindexes(toBeMerged, delete = False):
    """
    Requires a list of rasterio.open() objects.
    """
    for index in toBeMerged:
        src = toBeMerged[index][0]
        rasters = []
        print('Merging : ' + index)
        rasters.extend(toBeMerged[index])
        if exists(SHAREdir + '\\' + index + '_merged.tif'):
            previousMerge = rasterio.open(SHAREdir + '\\' + index + '_merged.tif')
            rasters.append(previousMerge)
        mosaic, out_trans = merge(rasters)
        
        kwargs = src.meta.copy()
        kwargs.update(
            driver='GTiff',
            height= mosaic.shape[1],
            width=mosaic.shape[2],
            transform=out_trans,
            dtype=rasterio.float32,
            count=1,
            compress='lzw')
        with rasterio.open(SHAREdir+'\\'+index+'_merged.tif', 'w', **kwargs) as dest:
            dest.write(mosaic)
    
    src.close()
    if 'previousMerge' in locals():
        previousMerge.close()
        
    for key in toBeMerged:
        for product in toBeMerged[key]:
            product.close()
    
def updateStructure(product, band, lastRefresh):
    """
    """
    os.chdir(HOMEdir)
    with open('structure.json', 'r') as json_file:
        structure = json.load(json_file)
    structure['lastRefresh'] = str(lastRefresh)
    newRast = rasterio.open(band)
    bboxFound = False
    i = 1
    for raster in structure['rasters']:
        if bboxFound == False:
            if raster['crs'] == str(newRast.crs) and raster['bbox'] == str(newRast.bounds):
                structure['rasters'][i-1]['product'] = product
                bboxFound = True
            i = i + 1
        else:
            break
    if bboxFound == False:
        structure['rasters'].append({'id':i, 'product': product, 'crs': str(newRast.crs), 'bbox': str(newRast.bounds)})
    
    with open('structure.json', 'w') as json_out:
        json.dump(structure, json_out)

    with open(HOMEdir + '\\structureArchive\\structure_' + str(dt.datetime.today().strftime('%Y%m%d')) + '.json', 'w') as json_out:
        json.dump(structure, json_out)
def masking(product, directory, maskingShp):
    """
    Mask product with a shapefile
    """
    print('Masking: ' + product)
    with fiona.open(maskingShp, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
    
    with rasterio.open(directory + '\\' + product) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True, nodata=np.nan, all_touched=True)
        out_meta = src.meta
    
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

    with rasterio.open(directory + '\\' + product.split('.tif')[0] + '_masked.tif', "w", **out_meta) as dest:
        dest.write(out_image)

## Water quality indexes

In [19]:
def cdom(product, B03, B04):
    """
    Calculates CDOM index.
    Requires the title of the product, b03 and b04 bands.
    Saves the CDOM index in results directiory.
    """
    print('    Calculating CDOM for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB03 = rasterio.open(B03)
    B03 = srcB03.read().astype('f4')
    B04 = rasterio.open(B04).read().astype('f4')

    B03[B03 <= 0] = np.nan
    B04[B04 <= 0] = np.nan

    cdom = 537 * np.exp(-2.93*B03/B04)

    cdom[cdom <= 0] = np.nan

    dst_crs = 'EPSG:3857'
    transform, wid, hei = calculate_default_transform(srcB03.crs, dst_crs, srcB03.width, srcB03.height, *srcB03.bounds)
    kwargs = srcB03.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('CDOM_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(cdom.astype(rasterio.float32))

    os.chdir(HOMEdir)
def turbidity(product, B03, B01):
    """
    Calculates turbidity.
    Requires the title of the product, b03 and b04 bands.
    Saves the turbidity raster in results directiory.
    """
    print('    Calculating turbidity for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB03 = rasterio.open(B03)
    B03 = srcB03.read().astype('f4')
    B01 = rasterio.open(B01).read().astype('f4')

    B03[B03 <= 0] = np.nan
    B01[B01 <= 0] = np.nan

    turb = 8.93 * (B03/B01) - 6.39

    #turb[turb <= 0] = np.nan

    kwargs = srcB03.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('turbidity_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(turb.astype(rasterio.float32))
    os.chdir(HOMEdir)
def doc(product, B03, B04):
    """
    Dissolved Organic Carbon

    """
    print('    Calculating DOC for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB03 = rasterio.open(B03)
    B03 = srcB03.read().astype('f4')
    B04 = rasterio.open(B04).read().astype('f4')

    B03[B03 <= 0] = np.nan
    B04[B04 <= 0] = np.nan

    doc = 432 * np.exp(-2.24*B03/B04)

    doc[doc <= 0] = np.nan

    kwargs = srcB03.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('DOC_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(doc.astype(rasterio.float32))
    os.chdir(HOMEdir)
def chl_a(product, B03, B01):
    """
    Concentration of Chlorophyll a
    """
    print('    Calculating Chl a for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB03 = rasterio.open(B03)
    B03 = srcB03.read().astype('f4')
    B01 = rasterio.open(B01).read().astype('f4')

    B03[B03 <= 0] = np.nan
    B01[B01 <= 0] = np.nan

    chla = 4.26 * np.float_power(B03/B01, 3.94)

    chla[chla <= 0] = np.nan

    kwargs = srcB03.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('chla_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(chla.astype(rasterio.float32))
    os.chdir(HOMEdir)
def cya(product, B03, B04, B02):
    """
    Density of Cyanobacteria
    """
    print('    Calculating Cya for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB03 = rasterio.open(B03)
    B03 = srcB03.read().astype('f4')
    B04 = rasterio.open(B04).read().astype('f4')
    B02 = rasterio.open(B02).read().astype('f4')

    B03[B03 <= 0] = np.nan
    B04[B04 <= 0] = np.nan
    B02[B02 <= 0] = np.nan

    cyaTemp = 115530.31 * np.float_power(B03*B04/B02, 2.38)
    cya = np.divide(cyaTemp,10**12)
    
    cya[cya <= 0] = np.nan

    kwargs = srcB03.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('cya_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(cya.astype(rasterio.float32))
    os.chdir(HOMEdir)    

## Drought indexes functions

In [20]:
def vci(product, B04, B08, histNDVI):
    """
    Calculates VCI index.
    Requires the title of the product, b04 and b08 bands with resolution 10m.
    Saves the VCI index in results directiory.
    """
    print('    Calculating VCI for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB04 = rasterio.open(B04)
    B04 = srcB04.read().astype('f4')
    B08 = rasterio.open(B08).read().astype('f4')

    B04[B04 <= 0] = np.nan
    B08[B08 <= 0] = np.nan
    
# ndvi matrix is required for calculating vci 
    try:
        ndvi = rasterio.open(histNDVI).read().astype('f4')
        ndviMax = np.nanmax(ndvi)
        ndviMin = np.nanmin(ndvi)
    except:
        print("VCI needs calculated historical NDVI")
    
    ndvi = np.divide((B08 - B04), (B08 + B04))
    vci = np.divide((ndvi - ndviMin), (ndviMax - ndviMin)) * 100

    vci[vci < 0] = np.nan
    vci[vci > 100] = np.nan

    dst_crs = 'EPSG:3857'
    transform, wid, hei = calculate_default_transform(srcB04.crs, dst_crs, srcB04.width, srcB04.height, *srcB04.bounds)
    kwargs = srcB04.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('VCI_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(vci.astype(rasterio.float32))

    os.chdir(HOMEdir)
    
### --------------------------------------------------------------------------------------------------------------------------------------------
def ndwi(product, B03, B08):
    """
    Calculates NDWI index.
    Requires the title of the product, b03 and b08 bands with resolution 10m.
    Saves the NDWI index in results directiory.
    """
    print('    Calculating NDWI for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB03 = rasterio.open(B03)
    B03 = srcB03.read().astype('f4')
    B08 = rasterio.open(B08).read().astype('f4')

    B03[B03 <= 0] = np.nan
    B08[B08 <= 0] = np.nan
    
    ndwi = np.divide((B03 - B08),(B03 + B08))

    ndwi[ndwi < -1] = np.nan
    ndwi[ndwi > 1] = np.nan

    dst_crs = 'EPSG:3857'
    transform, wid, hei = calculate_default_transform(srcB03.crs, dst_crs, srcB03.width, srcB03.height, *srcB03.bounds)
    kwargs = srcB03.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('NDWI_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(ndwi.astype(rasterio.float32))

    os.chdir(HOMEdir)
### --------------------------------------------------------------------------------------------------------------------------------------------
def nmdi(product, B08, B11, B12):
    """
    Calculates NMDI index.
    Requires the title of the product, b08, b11 and b12 bands with resolution 20m.
    Saves the NMDI index in results directiory.
    """
    print('    Calculating NMDI for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB11 = rasterio.open(B11) 
    B11 = srcB11.read().astype('f4')
    B12 = rasterio.open(B12).read().astype('f4')
    
# band 8 is not available in 20m, using 10m and resampling
    upscale_factor = 1/2
    with rasterio.open(B08) as B08init:
        # resample data to target shape
        B08 = B08init.read(
            out_shape=(
                B08init.count,
                int(B08init.height * upscale_factor),
                int(B08init.width * upscale_factor)
            ),
            resampling=Resampling.bilinear
        ).astype('f4')

    B08[B08 <= 0] = np.nan
    B11[B11 <= 0] = np.nan
    B12[B12 <= 0] = np.nan
    
    B11B12 = B11 - B12
    
    nmdi = np.divide(B08 - B11B12, B08 + B11B12)

#normalized, values from 0 to 1 are the only reasonable
    nmdi[nmdi < 0] = np.nan
    nmdi[nmdi > 1] = np.nan

    dst_crs = 'EPSG:3857'
    transform, wid, hei = calculate_default_transform(srcB11.crs, dst_crs, srcB11.width, srcB11.height, *srcB11.bounds)
    kwargs = srcB11.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('NMDI_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(nmdi.astype(rasterio.float32))

    os.chdir(HOMEdir)
### --------------------------------------------------------------------------------------------------------------------------------------------
def ndmi(product, B08, B11):
    """
    Calculates NDMI index.
    Requires the title of the product, b08 and b11 bands with resolution 20m.
    Saves the NDMI index in results directiory.
    """
    print('    Calculating NDMI for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB11 = rasterio.open(B11) 
    B11 = srcB11.read().astype('f4')
    
# band 8 is not available in 20m, using 10m and resampling
    upscale_factor = 1/2
    with rasterio.open(B08) as B08init:
        # resample data to target shape
        B08 = B08init.read(
            out_shape=(
                B08init.count,
                int(B08init.height * upscale_factor),
                int(B08init.width * upscale_factor)
            ),
            resampling=Resampling.bilinear
        ).astype('f4')

    B08[B08 <= 0] = np.nan
    B11[B11 <= 0] = np.nan
    
    ndmi = np.divide((B08 - B11), (B08 + B11))

    ndmi[ndmi < 0] = np.nan
    ndmi[ndmi > 1] = np.nan

    dst_crs = 'EPSG:3857'
    transform, wid, hei = calculate_default_transform(srcB11.crs, dst_crs, srcB11.width, srcB11.height, *srcB11.bounds)
    kwargs = srcB11.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('NDMI_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(ndmi.astype(rasterio.float32))

    os.chdir(HOMEdir)
### --------------------------------------------------------------------------------------------------------------------------------------------
def ndvi(product, B04, B08):
    """
    Calculates NDVI index.
    Requires the title of the product, b04 and b08 bands with resolution 20m.
    Saves the NDMI index in results directiory.
    """
    print('    Calculating NDVI for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB04 = rasterio.open(B04) 
    B04 = srcB04.read().astype('f4')
    B08 = rasterio.open(B08).read().astype('f4')

    B08[B08 <= 0] = np.nan
    B04[B04 <= 0] = np.nan
    
    ndvi = np.divide((B08 - B04), (B08 + B04))

    ndvi[ndvi < -1] = np.nan
    ndvi[ndvi > 1] = np.nan

    dst_crs = 'EPSG:3857'
    transform, wid, hei = calculate_default_transform(srcB04.crs, dst_crs, srcB04.width, srcB04.height, *srcB04.bounds)
    kwargs = srcB04.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('NDVI_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(ndvi.astype(rasterio.float32))

    os.chdir(HOMEdir)
### --------------------------------------------------------------------------------------------------------------------------------------------
def wdrvi(product, B04, B08):
    """
    Calculates WDRVI index.
    Requires the title of the product, b04 and b08 bands with resolution 20m.
    Saves the WDRVI index in results directiory.
    """
    print('    Calculating WDRVI for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB04 = rasterio.open(B04) 
    B04 = srcB04.read().astype('f4')
    B08 = rasterio.open(B08).read().astype('f4')

    B08[B08 <= 0] = np.nan
    B04[B04 <= 0] = np.nan
    
    wdrvi = np.divide(((0.1*B08) - B04), ((0.1*B08) + B04))

    #wdrvi[ndvi < -1] = np.nan
    #wdrvi[ndvi > 1] = np.nan

    dst_crs = 'EPSG:3857'
    transform, wid, hei = calculate_default_transform(srcB04.crs, dst_crs, srcB04.width, srcB04.height, *srcB04.bounds)
    kwargs = srcB04.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('WDRVI_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(wdrvi.astype(rasterio.float32))

    os.chdir(HOMEdir)
### --------------------------------------------------------------------------------------------------------------------------------------------
def evi(product, B02, B04, B08):
    """
    Calculates EVI index.
    Requires the title of the product, b04 and b08 bands with resolution 10m.
    Saves the EVI index in results directiory.
    """
    print('    Calculating EVI for', product)
# opening one of bands in separated to retrieve metadata to later save the raster
    srcB04 = rasterio.open(B04)
    B04 = srcB04.read().astype('f4')
    B08 = rasterio.open(B08).read().astype('f4')
    B02 = rasterio.open(B02).read().astype('f4')

    B04[B04 <= 0] = np.nan
    B08[B08 <= 0] = np.nan
    B02[B02 <= 0] = np.nan
    
    evi = np.divide((B08 - B04), (B08 + 6*B04 - 7.5*B02 +1)) * 2.5

    # evi[evi < 0] = np.nan
    # evi[evi > 100] = np.nan

    dst_crs = 'EPSG:3857'
    transform, wid, hei = calculate_default_transform(srcB04.crs, dst_crs, srcB04.width, srcB04.height, *srcB04.bounds)
    kwargs = srcB04.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('EVI_' + product + '.tif', 'w', **kwargs) as dst:
        dst.write(evi.astype(rasterio.float32))

    os.chdir(HOMEdir)

## Execution

In [5]:
products = dataCheck2A()

Products found: 
Found:  S2A_MSIL2A_20220404T084601_N0400_R107_T36UXB_20220404T125037.SAFE
Found:  S2A_MSIL2A_20220404T084601_N0400_R107_T36UYB_20220404T125037.SAFE


In [9]:
lastRefresh = dt.datetime.now()
if not products.empty:
    #for product in products.iterrows():
    dataDownload2A(products)
fileRefresh = open('lastRefresh.txt','w')
fileRefresh.write(str(lastRefresh))
fileRefresh.close()

Now downloading:  S2A_MSIL2A_20220404T084601_N0400_R107_T36UXB_20220404T125037.SAFE
Zipped file extracted to S2A_MSIL2A_20220404T084601_N0400_R107_T36UXB_20220404T125037.SAFE folder
Now downloading:  S2A_MSIL2A_20220404T084601_N0400_R107_T36UYB_20220404T125037.SAFE


MD5 checksumming:   0%|          | 0.00/381M [00:00<?, ?B/s]

Zipped file extracted to S2A_MSIL2A_20220404T084601_N0400_R107_T36UYB_20220404T125037.SAFE folder


In [24]:
# detecting if we are calculating hist data 
# then working on every product folder in DATAdir
if list(indexes.keys())[0].endswith("hist"):
    for file in os.listdir(DATAdir):
        if not file.endswith(".zip"):
            bands = bands2A(file)
# working on products from Open Access Hub
else:
    if not products.empty:
        for product in products.iterrows():
            bands = bands2A(product[1]['filename'])
            #showBand(bands['b02_10m'],'viridis')        
    # drought indexes
            vci(product[1]['title'], bands['b04_10m'], bands['b08_10m'], SHAREdir + '\\' + oblast + 'NDVIhist_merged_masked.tif')
            ndvi(product[1]['title'], bands['b04_10m'], bands['b08_10m'])
            evi(product[1]['title'], bands['b02_10m'], bands['b04_10m'], bands['b08_10m'])
            wdrvi(product[1]['title'], bands['b04_10m'], bands['b08_10m'])

            updateStructure(product[1]['title'], bands['b03_10m'], lastRefresh)

print('Reprojecting indexes in RESULTSdir to EPSG:3857 and saving to REPROJdir')
print('    Also removing files in RESULTSdir')
for file in os.listdir(RESULTSdir):
    if file.endswith('.tif'):
        epsg3857(file, RESULTSdir, True)

        
# from this part, indexes variable is needed to specify which indexes should be continued

# indexes = {'VCI':[], 'NDWI':[], 'NMDI':[]}
# indexes = {'NDMI':[], 'NDVI':[]}

for file in os.listdir(REPROJdir):
    if file.endswith('.tif'):
        param = file.split('_')[0]
        src = rasterio.open(REPROJdir + '\\' + file)
        indexes[param].append(src)
mergeWQindexes(indexes, True)

# this is required to close all open files before deleting files in REPROJdir
src.close()
for key in indexes:
    for product in indexes[key]:
        product.close()

print('Removing files from REPROJdir')
for filename in os.listdir(REPROJdir):
    if os.path.exists(REPROJdir + '\\' + filename):
        os.remove(REPROJdir + '\\' + filename)
        print('    Deleted: ' + filename)

for file in os.listdir(SHAREdir):
    if file.endswith('_merged.tif'):
        maskingShp = HOMEdir + "\\oblastExtent\\" + oblast + ".shp"
        masking(file, SHAREdir, maskingShp)

Getting bands directories for S2B_MSIL1C_20210411T083559_N0300_R064_T36UYA_20210411T104817.SAFE
None
Getting bands directories for S2B_MSIL1C_20210411T083559_N0300_R064_T36UYV_20210411T104817.SAFE
None


## Playground

In [8]:
lastRefresh = dt.datetime.now()
if not products.empty:
    for product in products.iterrows():
        bands = bands2A(product)
        #updateStructure(product[1]['title'], bands['b03_10m'], lastRefresh)

Getting bands directories for S2A_MSIL2A_20220317T024551_N0400_R132_T49MCM_20220317T065540.SAFE
Getting bands directories for S2A_MSIL2A_20220317T024551_N0400_R132_T49MCN_20220317T065540.SAFE


In [24]:
    srcB04 = rasterio.open(bands['b04_10m'])
    B04 = srcB04.read().astype('f4')
    B08 = rasterio.open(bands['b08_10m']).read().astype('f4')

    B04[B04 <= 0] = np.nan
    B08[B08 <= 0] = np.nan
    
# ndvi matrix is required for calculating vci 
    ndvi = np.divide((B08 - B04), (B08 + B04))
    print(ndvi)
    ndviMax = np.nanmax(ndvi)
    print(ndviMax)
    ndviMin = np.nanmin(ndvi)
    
    vci = np.divide((ndvi - ndviMin), (ndviMax - ndviMin)) * 100
    print(vci)

    dst_crs = 'EPSG:3857'
    transform, wid, hei = calculate_default_transform(srcB04.crs, dst_crs, srcB04.width, srcB04.height, *srcB04.bounds)
    kwargs = srcB04.meta.copy()
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    os.chdir(RESULTSdir)
    with rasterio.open('VCI_' + '.tif', 'w', **kwargs) as dst:
        dst.write(vci.astype(rasterio.float32))

    os.chdir(HOMEdir)

0.89109117
[[[36.92381  36.867992 36.587517 ... 36.69629  37.46336  36.831482]
  [35.992954 36.826145 37.351723 ... 37.529636 37.774456 36.437176]
  [35.46355  36.002636 37.3544   ... 35.60391  35.71861  35.890633]
  ...
  [40.02536  40.19489  40.51413  ... 57.798416 57.009876 55.562454]
  [39.74128  39.270016 39.36764  ... 60.53394  56.88607  50.559055]
  [38.383327 37.99328  38.425194 ... 65.97236  60.439278 50.62949 ]]]


In [23]:
ndviMax = np.nanmax(ndvi)
print(ndviMax)

b = np.arange(5, dtype=float)
print(b)
b[2] = np.NaN
print(b)
print(np.amax(b))
print(np.amax(b, where=~np.isnan(b), initial=-1))
print(np.nanmax(b))

0.89109117
[0. 1. 2. 3. 4.]
[ 0.  1. nan  3.  4.]
nan
4.0
4.0


In [8]:
for file in os.listdir(SHAREdir):
    if file.endswith('_merged.tif'):
        if file.endswith('NDWI_merged.tif') or file.endswith('VCI_merged.tif') or file.endswith('NMDI_merged.tif'):
            maskingShp = HOMEdir + "\\droughtExtent\\DiengPlateau.shp"
            masking(file, SHAREdir, maskingShp)

Masking: NDWI_merged.tif
Masking: NMDI_merged.tif
Masking: VCI_merged.tif


In [18]:
for file in os.listdir(RESULTSdir):
    if file.endswith('.tif'):
        epsg3857(file, RESULTSdir, True)

Reprojecting: CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T33UYR_20211024T122307
File CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T33UYR_20211024T122307.tif was deleted
Reprojecting: CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T34UCA_20211024T122307
File CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T34UCA_20211024T122307.tif was deleted
Reprojecting: CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T34UCV_20211024T122307
File CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T34UCV_20211024T122307.tif was deleted
Reprojecting: CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T34UDA_20211024T122307
File CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T34UDA_20211024T122307.tif was deleted
Reprojecting: CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T34UDV_20211024T122307
File CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T34UDV_20211024T122307.tif was deleted
Reprojecting: CDOM_S2B_MSIL2A_20211023T093039_N0301_R136_T34UEV_20211023T115253
File CDOM_S2B_MSIL2A_20211023T093039_N0301_R136_T34UEV_20211023T115253.tif was delete

In [4]:
indexes = {'CDOM':[],'chla':[],'cya':[],'DOC':[],'turbidity':[]}

for file in os.listdir(REPROJdir):
    if file.endswith('.tif'):
        param = file.split('_')[0]
        src = rasterio.open(REPROJdir + '\\' + file)
        indexes[param].append(src)
mergeWQindexes(indexes, True)

# this is required to close all open files before deleting files in REPROJdir
src.close()
for key in indexes:
    for product in indexes[key]:
        product.close()
        
for filename in os.listdir(REPROJdir):
    if os.path.exists(REPROJdir + '\\' + filename):
        os.remove(REPROJdir + '\\' + filename)
        print('    Deleted: ' + filename)

Merging : CDOM
Merging : chla
Merging : cya
Merging : DOC
Merging : turbidity


In [6]:
for file in os.listdir(SHAREdir):
    if file.endswith('_merged.tif'):
        masking(file, SHAREdir)

In [15]:
if not products.empty:
    for product in products.iterrows():
        bands = bands2A(product)
        #showBand(bands['b02_10m'],'viridis')
        vci(product[1]['title'], bands['b04_10m'], bands['b08_10m'])
        # ndwi(product[1]['title'], bands['b03_10m'], bands['b08_10m'])
        # nmdi(product[1]['title'], bands['b08_10m'], bands['b11_20m'], bands['b12_20m'])
        
# upscale_factor = 1/2
# with rasterio.open(bands['b08_10m']) as B08init:
#         # resample data to target shape
#         B08 = B08init.read(
#             out_shape=(
#                 B08init.count,
#                 int(B08init.height * upscale_factor),
#                 int(B08init.width * upscale_factor)
#             ),
#             resampling=Resampling.bilinear
#         ).astype('f4')

# B11 = rasterio.open(bands['b11_20m']).read().astype('f4')
# B12 = rasterio.open(bands['b12_20m']).read().astype('f4')

# print(B08.shape)
# print(B11.shape)
# print(B12.shape)

Getting bands directories for S2A_MSIL2A_20220317T024551_N0400_R132_T49MCM_20220317T065540.SAFE


TypeError: 'numpy.ndarray' object is not callable

In [40]:
print(data)

[[[1336 1332 1333 ... 2372 2495 1929]
  [1288 1333 1303 ... 2455 2416 1986]
  [1371 1354 1336 ... 2424 2475 2304]
  ...
  [4875 5258 6115 ... 4213 2896 2368]
  [5672 5761 6332 ... 4324 2390 2729]
  [6206 6263 5945 ... 3879 3234 2604]]]


In [26]:
if not products.empty:
    for product in products.iterrows():
        bands = bands2A(product)
        vci(product[1]['title'], bands['b04_10m'], bands['b08_10m'])

Getting bands directories for S2A_MSIL2A_20220317T024551_N0400_R132_T49MCM_20220317T065540.SAFE
    Calculating VCI for S2A_MSIL2A_20220317T024551_N0400_R132_T49MCM_20220317T065540
Getting bands directories for S2A_MSIL2A_20220317T024551_N0400_R132_T49MCN_20220317T065540.SAFE
    Calculating VCI for S2A_MSIL2A_20220317T024551_N0400_R132_T49MCN_20220317T065540


In [9]:
print(os.getcwd())
#print(rasters)

C:\Users\Asus\OneDrive - Stowarzyszenie IAESTE Polska\Studia\GINinzynierka\GINinz


In [22]:
# from arcgis.gis import GIS
# from arcgis.raster.analytics import copy_raster
import arcpy
# print(os.environ['PATH'])

In [ ]:
# Sign in to portal
arcpy.SignInToPortal("https://www.arcgis.com", "zuluzi", "Jem987dzem!")

# Set output file names
outdir = r"C:\Project\Output"
service_name = "TileSharingDraftExample"
sddraft_filename = service_name + ".sddraft"
sddraft_output_filename = os.path.join(outdir, sddraft_filename)
sd_filename = service_name + ".sd"
sd_output_filename = os.path.join(outdir, sd_filename)

# Reference map to publish
aprx = arcpy.mp.ArcGISProject(r"C:\Project\World.aprx")
m = aprx.listMaps('World')[0]

# Create TileSharingDraft and set metadata and portal folder properties
server_type = "HOSTING_SERVER"
sddraft = m.getWebLayerSharingDraft(server_type, "TILE", service_name)
sddraft.credits = "These are credits"
sddraft.description = "This is description"
sddraft.summary = "This is summary"
sddraft.tags = "tag1, tag2"
sddraft.useLimitations = "These are use limitations"
sddraft.portalFolder = "my folder name"

# Create Service Definition Draft file
sddraft.exportToSDDraft(sddraft_output_filename)

# Stage Service
print("Start Staging")
arcpy.StageService_server(sddraft_output_filename, sd_output_filename)

# Share to portal
print("Start Uploading")
arcpy.UploadServiceDefinition_server(sd_output_filename, server_type)

print("Finish Publishing")


In [6]:
# sprawdzenie algorytmu działającego pod merge
rasterss = []
rasterss.append(rasterio.open(REPROJdir + '\\cya_S2A_MSIL2A_20211024T095101_N0301_R079_T34UCA_20211024T122307_reprojected.tif'))
rasterss.append(rasterio.open(REPROJdir + '\\CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T33UYR_20211024T122307_reprojected.tif'))

src = rasterss[0]
mosaic, out_trans = merge(rasterss)
kwargs = src.meta.copy()
kwargs.update(
    driver='GTiff',
    height= mosaic.shape[1],
    width=mosaic.shape[2],
    transform=out_trans,
    dtype=rasterio.float32,
    count=1,
    compress='lzw')
with rasterio.open(SHAREdir+'\\test_merged.tif', 'w', **kwargs) as dest:
    dest.write(mosaic)

In [ ]:
gis = GIS(username="zuluzi", password="Jem987dzem!")

In [ ]:
my_content = gis.content.search(query="")

my_content

In [ ]:
input_raster_path = r"C:\Users\Asus\Documents\CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T33UYR_20211024T122307.tif"
single_image_layer = copy_raster(input_raster=input_raster_path,
                                 output_name="output_imagery_layer",
                                 gis=gis)

In [ ]:
results = os.listdir(RESULTSdir)
print(RESULTSdir + '\\' + results[0])

In [ ]:
os.environ['PATH'] = r';C:\Program Files\PostgreSQL\14\bin'
os.environ['PGHOST'] = 'localhost'
os.environ['PGPORT'] = '5432'
os.environ['PGUSER'] = 'postgres'
os.environ['PGPASSWORD'] = 'postgres'
os.environ['PGDATABASE'] = 'WQ'

results = os.listdir(RESULTSdir)

conn = psycopg2.connect(database="WQ", user="postgres", host="localhost", password="postgres")
cursor = conn.cursor()
cmds = 'raster2pgsql -t 100x100 "' + RESULTSdir + '\\' + results[0] + '" cdomrast.cdom1 |psql'

sql = "INSERT INTO rasters.cdom VALUES (st_raster(" + RESULTSdir + '\\' + results[0] + ",'compression=LZW'));"

subprocess.call(sql, shell=True)

conn.close()

In [ ]:
conn.close()

In [ ]:
if not products.empty:
    for product in products.iterrows():
        bands = bands2A(product)
for band in bands.keys():
    print(f'{band}: {bands[band]}')
for product in products.iterrows():
    for v1 in product[1].keys():
        print(f'{v1}: {product[1][v1]}')

In [ ]:
try1 = geojson_to_wkt(read_geojson('malopolska.geojson'))

In [ ]:
    srcB03 = rasterio.open(bands['b02_10m'])
    kwargs = srcB03.meta
    print(type(kwargs))
    print(kwargs['crs'])

In [ ]:
#B04 = rasterio.open(bands['b04_10m']).read(1).astype('f4')
#print(B04.shape[1])
#i = 0
#if B04[B04 <= 0] <= 0:
#    i = i+1
#print(i)


In [ ]:
for product in products.iterrows():
    prodBands = bands2A(product)
    CDOM(product[1]['title'], prodBands['B03'],prodBands['B04'])

In [ ]:
src = rasterio.open(RESULTSdir+r"\test123.tif")
kwargs = src.meta
print(kwargs)
srcB03 = rasterio.open(prodBands['B03'])
kwargs = srcB03.meta
print(kwargs)

In [ ]:
os.chdir(DATAdir) # get into data folder
for product in products_df.iterrows():
    try:
        prodBands = bandsDir(product)
        rgb = RGB(prodBands['REDdir'],prodBands['BLUEdir'],prodBands['GREENdir'])
        plt.imshow(rgb, cmap='RdYlGn')
        plt.colorbar()
        plt.title('RGB')
        plt.xlabel('Column #')
        plt.ylabel('Row #')
        os.chdir(DATAdir) # back to data folder
    except NameError:
        print('Error occurred while processing {0}'.format(product[1]['filename']))
os.chdir(HOMEdir) # back to main directory

In [ ]:
# The grid of raster values can be accessed as a numpy array and plotted:
with rasterio.open(prodBands['REDdir']) as src:
    print(type(src.read()[0][0][0]))
    print(src.read()[0][0][0])
    oviews = src.overviews(1) # list of overviews from biggest to smallest
    print(oviews)
    oview = oviews[-1] # let's look at the smallest thumbnail
    print(oview)
    print('Decimation factor= {}'.format(oview))
    # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
    thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

In [ ]:
products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])
products_df_sorted = products_df_sorted.head(1)
#print(products_df_sorted.uuid[0])
api.get_product_odata(products_df_sorted.uuid[0], full=True)
#api.download(products_df_sorted.uuid[0])

In [ ]:
odata = api.get_product_odata(products_df_sorted.uuid[0], full=True)
with zipfile.ZipFile(odata['title']+'.zip',"r") as zip_ref:
    zip_ref.extractall()

In [ ]:
#odata = api.get_product_odata(products_df_sorted.uuid[0], full=True)
#print(odata)
odata = api.get_product_odata(products_df_sorted.uuid[0], full=True)
print(odata['title'])
for row in products_df_sorted.head(1):
    print(row)

print(products_df_sorted.head(1).title)
#print(products_df_sorted.head(1))

In [ ]:
for product in products_df.iterrows():
        prodBands = bandsDir(product)
        print(prodBands)